# Analyse exploratoire


**1 - Importation des modules**

In [1]:
import pandas as pd
import numpy as np

from utils.data import get_gender
from utils.coherence import correct_typo
from utils.eda import apply_gender

**2 - Lecture des données**

In [2]:
df = pd.read_csv('df_patient_pcr.csv')

In [3]:
df.head()

,patient_id,pcr,given_name,surname,street_number,address_1,suburb,postcode,state,date_of_birth,age,phone_number,address_2,born_age,localisation,full_address,full_name
0,653091,N,daniel,campbell,58.0,sid barnes crescent,north ward,4514.0,nsw,19730426.0,NaN,08 38772117,NaN,19730426,4514 nsw north ward,58 sid barnes crescent,campbell daniel
1,347667,N,sebastian,mchenry,12.0,mundawari circuit,swan view,4551.0,wa,19900316.0,28.0,08 61083524,ocean star villas,19900316 28,4551 wa swan view,12 mundawari circuit,mchenry sebastian
2,708481,P,sarah,rellos,20.0,torres street,bribie island,3199.0,sa,19430317.0,NaN,02 73197286,NaN,19430317,3199 sa bribie island,20 torres street,rellos sarah
3,148825,N,chloe,brammy,238.0,fitchett street,carnegie,3280.0,qld,19540528.0,19.0,02 48826642,NaN,19540528 19,3280 qld carnegie,238 fitchett street,brammy chloe
4,150081,N,charlie,NaN,6.0,clark close,south melbourne,2602.0,qld,19750331.0,35.0,03 24096098,talawa,19750331 35,2602 qld south melbourne,6 clark close,charlie


## Pré-traitement des données

Avant de commencer l'analyse exploratoires, nous allons nettoyer la base de données et créer de nouvelles variables pouvant être pertinentes lors de l'analyse.  Enfin, nous sélectionnerons les variables pertinentes pour créer notre jeu de données final.

### Création d'une variable Sexe

On commence par corriger les prénom mal orthographié avec la même fonction utilisée précédemment pour les adresses et la banlieu. Après plusieurs itération, une similtude à 85% entre deux chaîne de caractère semble convenir en prenant comme base référence les prénoms apparaissant plus de 3 fois.

In [4]:
given_name_corrected = correct_typo(df.given_name, threshold=3,confidence=85)
df.given_name = df.given_name.replace(given_name_corrected)

On utilise ensuite une [API](https://genderize.io) qui fournit le sexe s'apparatant au mieux à un prénom, avec une probabilité indiquant la confiance de cette prédiction. Toutefois, l'API limitant ses requêtes, si l'exception est levée alors renvoie la dernière données sauvegardée.

In [5]:
unique_name = df.given_name.unique()
all_gender = get_gender(unique_name)

Request limit


In [7]:
apply_gender(all_gender, apply=df.given_name)

0         male
1         male
2       female
3       female
4         male
         ...  
7264      male
7265      male
7266      male
7267    female
7268    female
Name: given_name, Length: 7269, dtype: object